In [ ]:
pip install textblob

In [ ]:
pip install nltk

In [ ]:
pip install vaderSentiment

In [ ]:
from textblob import TextBlob
# VADER can be accessed by the NLTK library.
import nltk
# Download the VADAR tool and access it through the NLTK library.
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import tweepy
import re

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
feb_11_2021 = pd.read_csv('/content/drive/MyDrive/FMT 6/Data frames/11-02-2021.csv')
feb_11_2021.head(1)

,Unnamed: 0,_api,_json,created_at,id,id_str,text,source,source_url,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,author,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,matching_rules,display_text_range,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet
0,0,<tweepy.api.API object at 0x7f1009867e90>,{'created_at': 'Thu Feb 11 23:59:59 +0000 2021...,2021-02-11 23:59:59,1360015761188782080,1360015761188782080,RT @Ashwsbreal: #BreakingNews 📰 \n\n#Bitcoin O...,Twitter for iPhone,http://twitter.com/download/iphone,False,NaN,NaN,NaN,NaN,NaN,User(_api=<tweepy.api.API object at 0x7f100986...,User(_api=<tweepy.api.API object at 0x7f100986...,NaN,NaN,NaN,NaN,Status(_api=<tweepy.api.API object at 0x7f1009...,False,0,0,0,0,"{'hashtags': [{'text': 'BreakingNews', 'indice...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#cleaning the tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [ ]:
feb_11_2021['text'] = clean_tweets(feb_11_2021['text'])
feb_11_2021['text'].head()

0     #BreakingNews 📰 \n\n#Bitcoin On the Verge of ...
1    #Bitcoin is free market revolution without vio...
2                                            #Bitcoin 
3     Venezuela, entre los 10 países con mayor mine...
4     Someone just transfered 19,600 #bitcoin in on...
Name: text, dtype: object

In [ ]:
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(feb_11_2021['text'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(feb_11_2021['text'][i])["compound"]
    pos = analyzer.polarity_scores(feb_11_2021['text'][i])["pos"]
    neu = analyzer.polarity_scores(feb_11_2021['text'][i])["neu"]
    neg = analyzer.polarity_scores(feb_11_2021['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [ ]:
sentiments_score = pd.DataFrame.from_dict(scores)
feb_11_2021 = feb_11_2021.join(sentiments_score)
feb_11_2021.head()

,Unnamed: 0,_api,_json,created_at,id,id_str,text,source,source_url,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,author,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,matching_rules,display_text_range,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet,Compound,Positive,Negative,Neutral
0,0,<tweepy.api.API object at 0x7f1009867e90>,{'created_at': 'Thu Feb 11 23:59:59 +0000 2021...,2021-02-11 23:59:59,1360015761188782080,1360015761188782080,#BreakingNews 📰 \n\n#Bitcoin On the Verge of ...,Twitter for iPhone,http://twitter.com/download/iphone,False,NaN,NaN,NaN,NaN,NaN,User(_api=<tweepy.api.API object at 0x7f100986...,User(_api=<tweepy.api.API object at 0x7f100986...,NaN,NaN,NaN,NaN,Status(_api=<tweepy.api.API object at 0x7f1009...,False,0,0,0,0,"{'hashtags': [{'text': 'BreakingNews', 'indice...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.000,0.0,1.000
1,1,<tweepy.api.API object at 0x7f1009867e90>,{'created_at': 'Thu Feb 11 23:59:59 +0000 2021...,2021-02-11 23:59:59,1360015760635154434,1360015760635154434,#Bitcoin is free market revolution without vio...,Twitter for Android,http://twitter.com/download/android,False,NaN,NaN,NaN,NaN,NaN,User(_api=<tweepy.api.API object at 0x7f100986...,User(_api=<tweepy.api.API object at 0x7f100986...,NaN,NaN,NaN,NaN,NaN,False,0,0,0,1,"{'hashtags': [{'text': 'Bitcoin', 'indices': [...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7646,0.569,0.0,0.431
2,2,<tweepy.api.API object at 0x7f1009867e90>,{'created_at': 'Thu Feb 11 23:59:59 +0000 2021...,2021-02-11 23:59:59,1360015758751911939,1360015758751911939,#Bitcoin,Twitter for iPhone,http://twitter.com/download/iphone,False,NaN,NaN,NaN,NaN,NaN,User(_api=<tweepy.api.API object at 0x7f100986...,User(_api=<tweepy.api.API object at 0x7f100986...,NaN,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [{'text': 'Bitcoin', 'indices': [...",False,False,low,und,[{'tag': None}],"[0, 8]","{'media': [{'id': 1360015748798836737, 'id_str...",False,NaN,NaN,NaN,NaN,NaN,0.0000,0.000,0.0,1.000
3,3,<tweepy.api.API object at 0x7f1009867e90>,{'created_at': 'Thu Feb 11 23:59:58 +0000 2021...,2021-02-11 23:59:58,1360015756805746689,1360015756805746689,"Venezuela, entre los 10 países con mayor mine...",Twitter for Android,http://twitter.com/download/android,False,NaN,NaN,NaN,NaN,NaN,User(_api=<tweepy.api.API object at 0x7f100986...,User(_api=<tweepy.api.API object at 0x7f100986...,NaN,NaN,NaN,NaN,Status(_api=<tweepy.api.API object at 0x7f1009...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,es,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2023,0.079,0.0,0.921
4,4,<tweepy.api.API object at 0x7f1009867e90>,{'created_at': 'Thu Feb 11 23:59:57 +0000 2021...,2021-02-11 23:59:57,1360015752019857408,1360015752019857408,"Someone just transfered 19,600 #bitcoin in on...",Twitter Web App,https://mobile.twitter.com,False,NaN,NaN,NaN,NaN,NaN,User(_api=<tweepy.api.API object at 0x7f100986...,User(_api=<tweepy.api.API object at 0x7f100986...,NaN,NaN,NaN,NaN,Status(_api=<tweepy.api.API object at 0x7f1009...,False,0,0,0,0,"{'hashtags': [{'text': 'bitcoin', 'indices': [...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.000,0.0,1.000


In [ ]:
feb_11_2021.lang.value_counts()

en     470
es      10
und      7
ja       6
fr       3
tr       2
pt       1
de       1
Name: lang, dtype: int64

In [ ]:
feb_11_2021[['created_at','id','text','Compound','Positive','Negative','Neutral']]

,created_at,id,text,Compound,Positive,Negative,Neutral
0,2021-02-11 23:59:59,1360015761188782080,#BreakingNews 📰 \n\n#Bitcoin On the Verge of ...,0.0000,0.000,0.0,1.000
1,2021-02-11 23:59:59,1360015760635154434,#Bitcoin is free market revolution without vio...,0.7646,0.569,0.0,0.431
2,2021-02-11 23:59:59,1360015758751911939,#Bitcoin,0.0000,0.000,0.0,1.000
3,2021-02-11 23:59:58,1360015756805746689,"Venezuela, entre los 10 países con mayor mine...",0.2023,0.079,0.0,0.921
4,2021-02-11 23:59:57,1360015752019857408,"Someone just transfered 19,600 #bitcoin in on...",0.0000,0.000,0.0,1.000
...,...,...,...,...,...,...,...
495,2021-02-11 23:55:33,1360014642563670020,First #bitcoin etf,0.0000,0.000,0.0,1.000
496,2021-02-11 23:55:32,1360014638595801088,"Just another day, lol...\n- Twitter consideri...",0.4215,0.141,0.0,0.859
497,2021-02-11 23:55:32,1360014637702344708,"LADIES &amp; GENTS, WE HAVE THE 1ST PUBLICLY ...",0.0000,0.000,0.0,1.000
498,2021-02-11 23:55:32,1360014637304057856,"LADIES &amp; GENTS, WE HAVE THE 1ST PUBLICLY ...",0.0000,0.000,0.0,1.000


In [ ]:
feb_11_2021['compound'][1]

KeyError: ignored

In [ ]:
str(feb_11_2021['created_at'][0])[0:10]

'2021-02-11'

In [ ]:
#chipotle_df.item_price = chipotle_df.item_price.map(lambda n: float(n.replace('$','')))
feb_11_2021['date'] = feb_11_2021.created_at.map(lambda d: str(d)[0:10])

In [ ]:
feb_11_2021[['created_at','date']]

,created_at,date
0,2021-02-11 23:59:59,2021-02-11
1,2021-02-11 23:59:59,2021-02-11
2,2021-02-11 23:59:59,2021-02-11
3,2021-02-11 23:59:58,2021-02-11
4,2021-02-11 23:59:57,2021-02-11
...,...,...
495,2021-02-11 23:55:33,2021-02-11
496,2021-02-11 23:55:32,2021-02-11
497,2021-02-11 23:55:32,2021-02-11
498,2021-02-11 23:55:32,2021-02-11
